In [3]:
import os
import pandas as pd
import numpy as np
import joblib
import yaml
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Load config
with open("../config.yaml", "r") as f:
    config = yaml.safe_load(f)

# Paths
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
root_dir = os.path.abspath(os.path.join(notebook_dir, ".."))

preprocessed_path = os.path.join(root_dir, "data/processed/sample_preprocessed.csv")
models_dir = os.path.join(root_dir, "models")
results_dir = os.path.join(root_dir, "results")

target_col = config["data"]["target"]

# Load preprocessed data
df = pd.read_csv(preprocessed_path)
print("✅ Dataset loaded:", df.shape)

# Drop non-numeric columns
non_numeric_cols = df.select_dtypes(exclude=["number"]).columns
if len(non_numeric_cols) > 0:
    print(f"⚠ Dropping non-numeric columns: {list(non_numeric_cols)}")
    df = df.drop(columns=non_numeric_cols)

# Features and labels
X = df.drop(columns=[target_col])
y = pd.Categorical(df[target_col]).codes

# Train/Validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
print(f"Train shape: {X_train.shape}, Validation shape: {X_val.shape}")

✅ Dataset loaded: (100000, 72)
⚠ Dropping non-numeric columns: ['Source_File']
Train shape: (70000, 70), Validation shape: (30000, 70)


In [4]:
import time
import json

# --- Helper Function ---
def train_and_evaluate(X_train, y_train, X_val, y_val, scenario_name):
    print(f"\n🔹 Running scenario: {scenario_name}")
    start_time = time.time()
    try:
        # Train model
        model = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
        model.fit(X_train, y_train)
        print("✅ Model trained.")

        # Predict
        y_pred = model.predict(X_val)

        # Confusion Matrix (show first part only)
        cm = confusion_matrix(y_val, y_pred)
        print("Confusion Matrix (first 5x5 block):\n", cm[:5, :5])

        # Classification report
        report = classification_report(y_val, y_pred, output_dict=True)
        report_df = pd.DataFrame(report).transpose()
        print("\nClassification Report (head):")
        print(report_df.head())

        # Save report
        os.makedirs(results_dir, exist_ok=True)
        save_path = os.path.join(results_dir, f"bias_{scenario_name}.json")
        with open(save_path, "w") as f:
            json.dump(report, f, indent=4)
        print(f"✅ Metrics saved to {save_path}")

    except Exception as e:
        print(f"❌ Error in {scenario_name}: {e}")
    finally:
        print(f"⏱️ Time taken: {time.time() - start_time:.2f} seconds")


# --- Bias Scenario Simulation ---

# Gender bias → Assume rows 0-50% are "Male", 50-100% "Female"
split_idx = len(X_train) // 2
X_train_male, X_train_female = X_train.iloc[:split_idx], X_train.iloc[split_idx:]
y_train_male, y_train_female = y_train[:split_idx], y_train[split_idx:]

split_idx_val = len(X_val) // 2
X_val_male, X_val_female = X_val.iloc[:split_idx_val], X_val.iloc[split_idx_val:]
y_val_male, y_val_female = y_val[:split_idx_val], y_val[split_idx_val:]

# Region bias → Random split (simulating data from different regions)
X_train_eu, X_train_asia, _, _ = np.array_split(X_train, 4)
y_train_eu, y_train_asia, _, _ = np.array_split(y_train, 4)
X_val_eu, X_val_asia, _, _ = np.array_split(X_val, 4)
y_val_eu, y_val_asia, _, _ = np.array_split(y_val, 4)

# --- Run scenarios ---
scenarios = [
    ("gender_male_bias", X_train_male, y_train_male, X_val_male, y_val_male),
    ("gender_female_bias", X_train_female, y_train_female, X_val_female, y_val_female),
    ("region_eu_bias", X_train_eu, y_train_eu, X_val_eu, y_val_eu),
    ("region_asia_bias", X_train_asia, y_train_asia, X_val_asia, y_val_asia)
]

print("Sanity check: starting bias scenario run...")
for name, Xtr, ytr, Xv, yv in scenarios:
    print(f"\n--- {name.upper()} ---")
    train_and_evaluate(Xtr, ytr, Xv, yv, name)

print("\n✅ All bias scenarios completed successfully.")


C:\Users\bhand\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Sanity check: starting bias scenario run...

--- GENDER_MALE_BIAS ---

🔹 Running scenario: gender_male_bias
✅ Model trained.
Confusion Matrix (first 5x5 block):
 [[12580     0     0     1     2]
 [    6     5     0     0     0]
 [    1     0   706     0     0]
 [    1     0     0    56     0]
 [   11     0     0     0   969]]

Classification Report (head):
   precision    recall  f1-score  support
0   0.997858  0.999047  0.998452  12592.0
1   1.000000  0.454545  0.625000     11.0
2   1.000000  0.998586  0.999292    707.0
3   0.982456  0.982456  0.982456     57.0
4   0.997940  0.988776  0.993337    980.0
✅ Metrics saved to C:\Users\bhand\ids-bias-project\results\bias_gender_male_bias.json
⏱️ Time taken: 6.98 seconds

--- GENDER_FEMALE_BIAS ---

🔹 Running scenario: gender_female_bias
✅ Model trained.
Confusion Matrix (first 5x5 block):
 [[12548     2     0     1     4]
 [    9     3     0     0     0]
 [    2     0   747     0     0]
 [    3     0     0    55     0]
 [   11     0     0  

C:\Users\bhand\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\bhand\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\bhand\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave


Classification Report (head):
   precision    recall  f1-score  support
0   0.996664  0.998965  0.997813  12561.0
1   0.600000  0.250000  0.352941     12.0
2   1.000000  0.997330  0.998663    749.0
3   0.982143  0.948276  0.964912     58.0
4   0.994737  0.988494  0.991605    956.0
✅ Metrics saved to C:\Users\bhand\ids-bias-project\results\bias_gender_female_bias.json
⏱️ Time taken: 6.42 seconds

--- REGION_EU_BIAS ---

🔹 Running scenario: region_eu_bias
✅ Model trained.
Confusion Matrix (first 5x5 block):
 [[6284    0    1    0    2]
 [   4    3    0    0    0]
 [   0    0  374    0    0]
 [   0    0    0   25    0]
 [   5    0    0    0  484]]

Classification Report (head):
   precision    recall  f1-score  support
0   0.997619  0.998887  0.998253   6291.0
1   1.000000  0.428571  0.600000      7.0
2   0.997333  1.000000  0.998665    374.0
3   1.000000  1.000000  1.000000     25.0
4   0.995885  0.989775  0.992821    489.0
✅ Metrics saved to C:\Users\bhand\ids-bias-project\results\bias

C:\Users\bhand\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\bhand\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\bhand\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

✅ Model trained.
Confusion Matrix (first 5x5 block):
 [[6293    0    1    1    0]
 [   3    1    0    0    0]
 [   1    0  332    0    0]
 [   3    0    0   29    0]
 [   7    0    1    0  483]]

Classification Report (head):
   precision    recall  f1-score  support
0   0.997148  0.998730  0.997938   6301.0
1   1.000000  0.250000  0.400000      4.0
2   0.994012  0.996997  0.995502    333.0
3   0.966667  0.906250  0.935484     32.0
4   1.000000  0.983707  0.991786    491.0
✅ Metrics saved to C:\Users\bhand\ids-bias-project\results\bias_region_asia_bias.json
⏱️ Time taken: 2.91 seconds

✅ All bias scenarios completed successfully.


In [5]:
# Drop 80% of class 1
rare_class = 1
drop_frac = 0.8

mask = np.where(y_train == rare_class)[0]
drop_n = int(len(mask) * drop_frac)
drop_idx = np.random.choice(mask, drop_n, replace=False)

X_train_under = X_train.drop(X_train.index[drop_idx])
y_train_under = np.delete(y_train, drop_idx)

print(f"Dropped {drop_n} samples from class {rare_class}. New train shape: {X_train_under.shape}")

# Train
model_under = train_and_evaluate(X_train_under, y_train_under, X_val, y_val, "underrepresented_class1")


Dropped 40 samples from class 1. New train shape: (69960, 70)

🔹 Running scenario: underrepresented_class1
✅ Model trained.
Confusion Matrix (first 5x5 block):
 [[25133     0     1     2     2]
 [   17     6     0     0     0]
 [    3     0  1453     0     0]
 [    5     0     0   110     0]
 [   20     0     0     0  1916]]

Classification Report (head):
   precision    recall  f1-score  support
0   0.997500  0.999205  0.998352  25153.0
1   1.000000  0.260870  0.413793     23.0
2   0.999312  0.997940  0.998625   1456.0
3   0.982143  0.956522  0.969163    115.0
4   0.998957  0.989669  0.994292   1936.0
✅ Metrics saved to C:\Users\bhand\ids-bias-project\results\bias_underrepresented_class1.json
⏱️ Time taken: 16.66 seconds


C:\Users\bhand\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\bhand\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\bhand\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

In [6]:
# Drop 50% of class 0
major_class = 0
mask = np.where(y_train == major_class)[0]
drop_n = int(len(mask) * 0.5)
drop_idx = np.random.choice(mask, drop_n, replace=False)

X_train_imb = X_train.drop(X_train.index[drop_idx])
y_train_imb = np.delete(y_train, drop_idx)

print(f"Dropped {drop_n} samples from class {major_class}. New train shape: {X_train_imb.shape}")

# Train
model_imb = train_and_evaluate(X_train_imb, y_train_imb, X_val, y_val, "label_imbalance_class0")

Dropped 29286 samples from class 0. New train shape: (40714, 70)

🔹 Running scenario: label_imbalance_class0
✅ Model trained.
Confusion Matrix (first 5x5 block):
 [[25126     4     1     2     5]
 [   12    11     0     0     0]
 [    3     0  1453     0     0]
 [    4     0     0   111     0]
 [   15     0     0     0  1921]]

Classification Report (head):
   precision    recall  f1-score  support
0   0.998093  0.998927  0.998510  25153.0
1   0.733333  0.478261  0.578947     23.0
2   0.999312  0.997940  0.998625   1456.0
3   0.982301  0.965217  0.973684    115.0
4   0.997404  0.992252  0.994821   1936.0
✅ Metrics saved to C:\Users\bhand\ids-bias-project\results\bias_label_imbalance_class0.json
⏱️ Time taken: 6.27 seconds


C:\Users\bhand\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\bhand\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\bhand\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

In [7]:
# Flip 5% labels randomly
noise_frac = 0.05
n_noise = int(len(y_train) * noise_frac)
noise_idx = np.random.choice(len(y_train), n_noise, replace=False)

y_train_noisy = y_train.copy()
y_train_noisy[noise_idx] = np.random.randint(0, len(np.unique(y_train)), n_noise)

print(f"Added label noise to {n_noise} samples")

# Train
model_noise = train_and_evaluate(X_train, y_train_noisy, X_val, y_val, "label_noise_5pct")

Added label noise to 3500 samples

🔹 Running scenario: label_noise_5pct
✅ Model trained.
Confusion Matrix (first 5x5 block):
 [[25048    14    12     9     9]
 [   13    10     0     0     0]
 [    3     0  1451     0     1]
 [    4     0     0   108     0]
 [   16     2     2     0  1915]]

Classification Report (head):
   precision    recall  f1-score  support
0   0.997889  0.995826  0.996856  25153.0
1   0.333333  0.434783  0.377358     23.0
2   0.984396  0.996566  0.990444   1456.0
3   0.915254  0.939130  0.927039    115.0
4   0.993773  0.989153  0.991457   1936.0
✅ Metrics saved to C:\Users\bhand\ids-bias-project\results\bias_label_noise_5pct.json
⏱️ Time taken: 35.27 seconds
